Importing the libraries needed for the project, and reading in the first dataset (boardgames_ranks.csv)

In [1]:
import pandas as pd
import requests
import numpy as np
import xml.etree.ElementTree as ET
import time
import re

df_board_game_rankings = pd.read_csv('.\\Data_Sets\\boardgames_ranks.csv')

Data Read-in & Cleaning
1) Reading in the same dataset under a different variable name to preserve the original.
2) Filtering out board games with an overall rank of 0, as we only want games with an actual ranking.
3) Creating dataframe df_bgr_top which constrains boardgames to those with an overall rank of 1 - 250.
4) Replacing NaN values in the DataFrame with '0'
5) Renaming fields.
6) Converting field data types to necessary values.
7) Rounding averages to 2 decimal points.
8) Write file to csv for Tableau Visualization, no index.


In [2]:
##1
df_bgr2 = pd.read_csv('.\\Data_Sets\\boardgames_ranks.csv')

##2
zerorank = df_bgr2[(df_bgr2['rank'] == 0)].index
df_bgr_nozero = pd.DataFrame(df_bgr2.drop(zerorank, inplace = True))

##3
df_bgr_top = pd.DataFrame(df_bgr2.loc[(df_bgr2['rank'] >=1) & (df_bgr2['rank'] <=250)])

##4
df_bgr_top.fillna(0, inplace=True)

##5
df_bgr_top.rename(columns={'id': 'BGG_ID', 'name': 'GAME_NAME', 'yearpublished': 'PUBLISH_YR', 'rank': 'OVERALL_RANK', 'bayesaverage': 'BAYES_AVG',
                           'average': 'AVG_RATING','usersrated': 'USER_RATING', 'is_expansion': 'EXPANSION_IND', 'abstracts_rank': 'ABSTRACTS_RANK',
                           'cgs_rank': 'CGS_RANK', 'childrensgames_rank': 'CHILDREN_GAME_RANK', 'familygames_rank': 'FAMILY_RANK', 'partygames_rank': 'PARTY_RANK',
                           'strategygames_rank': 'STRATEGY_RANK', 'thematic_rank': 'THEMATIC_RANK', 'wargames_rank': 'WARGAME_RANK'}, inplace=True)

##6
convert_dict = {'BGG_ID': int,
                'GAME_NAME': object,
                'PUBLISH_YR': int,
                'OVERALL_RANK': int,
                'BAYES_AVG': float,
                'AVG_RATING': float,
                'USER_RATING': int,
                'EXPANSION_IND': bool,
                'ABSTRACTS_RANK': int,
                'CGS_RANK': int,
                'CHILDREN_GAME_RANK': int,
                'FAMILY_RANK': int,
                'PARTY_RANK': int,
                'STRATEGY_RANK': int,
                'THEMATIC_RANK': int,
                'WARGAME_RANK': int}
df_bgr_top = df_bgr_top.astype(convert_dict)

#7
df_bgr_top.BAYES_AVG = df_bgr_top.BAYES_AVG.round(2)
df_bgr_top.AVG_RATING = df_bgr_top.AVG_RATING.round(2)

#8
df_bgr_top.to_csv('.\\Data_Sets\\Tableau_Board_Game_Rankings.csv', index=False)


Extracting Information from BGG API, Merging Dataframes, Cleaning
1) Extracting list of 250 BGG_IDs to feed into BGG_XML_API_2
2) Setting up variables and list storage for the loop.
3) Iterate through all 250 BGG_IDs, and request XML for each. Sleeping when BGG_ID %7 is equal to 0, to adhere to API usage terms.
4) Parse XML for Mechanic Name and ID.
5) Extract all mechanics associated with a game, and append to a list.
6) Create pd.Series from lists.
7) Create pd.Dataframe from multiple series.
8) Convert field data types to necessary values.
9) Perform left join/merge between board game rankings dataframe and board game mechanics dataframe, on BGG_ID
10) Rounding averages to 2 places.
11) Write merged Dataframe to CSV, no index.

***NOTE - This block will take 6-7 minutes to run.

In [3]:
##1
bg_ids = df_bgr_top[str('BGG_ID')].values.tolist()

##2
API_base_string = 'https://boardgamegeek.com/xmlapi2/thing?id='
mech_id_ls = []
mech_name_ls = []
mech_bggid_ls = []

##3
for id in bg_ids:
    api_rec = requests.get(API_base_string + str(id))
    api_data = api_rec.content
    root = ET.fromstring(api_data)
    if id%4 == 0: 
      time.sleep(5)

##4,5
      for item in root:
        for link in item.findall('link'):
          if(link.get('type') == 'boardgamemechanic'):
            mech_id_ls.append(link.get('id'))
            mech_name_ls.append(link.get('value'))
            mech_bggid_ls.append(item.get('id'))
    else:
      for item in root:
        for link in item.findall('link'):
           if(link.get('type') == 'boardgamemechanic'):
            mech_id_ls.append(link.get('id'))
            mech_name_ls.append(link.get('value'))
            mech_bggid_ls.append(item.get('id'))

##6       
mech_bggid_ser = pd.Series(mech_bggid_ls)
mech_id_ser = pd.Series(mech_id_ls)
mech_name_ser = pd.Series(mech_name_ls)

##7
mechv3_frame = {'BGG_ID': mech_bggid_ser, 'MECH_ID': mech_id_ser, 'MECH_NAME': mech_name_ser}
df_mechv3 = pd.DataFrame(mechv3_frame)

##8
mech_convert_dict = {'BGG_ID': int,
                     'MECH_ID': object}
df_mechv3 = df_mechv3.astype(mech_convert_dict)

##9
df_gamemech_merge = pd.merge(df_bgr_top, df_mechv3,on='BGG_ID',how='left')
df_game_mech = pd.DataFrame(df_gamemech_merge)

##10
df_game_mech.BAYES_AVG = df_game_mech.BAYES_AVG.round(2)
df_game_mech.AVG_RATING = df_game_mech.AVG_RATING.round(2)

##11
df_game_mech.to_csv('.\\Data_Sets\\Tableau_Games_Mechanics.csv', index=False)




Create Pandas Pivot table from Mechanics
1) Create small pivot table to count recurrence of mechanics in top 250 board games
2) Flatten index/columns of pivot for easier manipulation
3) Rename Pivot Table columns
4) Write Pivot Table to csv, no index.
 

In [4]:
##1
df_gamemech_pivot = df_game_mech.pivot_table(values='BGG_ID', index=('MECH_NAME'),columns=None, aggfunc='count')

##2
flat_df_gamemech_pivot = df_gamemech_pivot.reset_index()

##3
name_dict = {'MECH_NAME': 'MECHANIC_NAME', 'BGG_ID': 'MECHANIC_COUNT'}
flat_df_gamemech_pivot.rename(columns=name_dict,inplace=True)
flat_df_gamemech_pivot.sort_values('MECHANIC_COUNT',ascending=False, inplace=True)

##4
flat_df_gamemech_pivot.to_csv('.\\Data_Sets\\Tableau_Game_Mechanic_Pivot.csv', index=False)


Designers
1) Read in Designers with Location CSV file.
2) Creating new dataframe constrained by only the BGG_IDs existing in the top 250.
3) Replace NaN values with 'UNK'.
4) Merge Game & Designer Dataframes
5) Output to CSV, no index.

In [5]:
##1
df_bgdesigner_loc = pd.read_csv('.\\Data_Sets\\BGG_Designer_Location.csv')

##2
df_topbgdes_loc = pd.DataFrame(df_bgdesigner_loc.loc[df_bgdesigner_loc['BGG_ID'].isin(bg_ids)])
df_topbgdes_loc.reset_index(drop=True, inplace=True)

##3
df_topbgdes_loc.fillna('UNKNOWN', inplace=True)

##4
df_game_des_merge = pd.merge(df_bgr_top, df_topbgdes_loc,on='BGG_ID',how='left')
df_game_des = pd.DataFrame(df_game_des_merge)

##5
df_game_des.to_csv('.\\Data_Sets\\Tableau_Games_Designer_Full.csv', index=False)

Designer Pivot Table

1) Create pivot from board game ranks and designer
2) Rename Columns
3) Sort Descending by Count
4) Output to CSV, no index.

In [7]:
##1
df_gamedes_pivot = df_game_des.pivot_table(values='BGG_ID', index=('DESIGNER_POB_CTRY'),columns=None, aggfunc='count')
flat_df_gamedes_pivot = df_gamedes_pivot.reset_index()

##2
name_dict = {'DESIGNER_COUNTRY': 'DESIGNER_COUNTRY', 'BGG_ID': 'DESIGNER_COUNT'}
flat_df_gamedes_pivot.rename(columns=name_dict,inplace=True)

##3
flat_df_gamedes_pivot.sort_values('DESIGNER_COUNT',ascending=False, inplace=True)

##4
flat_df_gamedes_pivot.to_csv('.\\Data_Sets\\Tableau_Game_Designer_Pivot.csv', index=False)
